#Deep Learning - Proyecto 1

In [4]:
from __future__ import division
from __future__ import print_function
from vizdoom import *
import itertools as it
from random import sample, randint, random
from time import time, sleep
import numpy as np
import skimage.color, skimage.transform
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from tqdm import trange
from torch.utils.data import IterableDataset, DataLoader
from torch.distributions import Categorical
# ⚡ PyTorch Lightning
import pytorch_lightning as pl

# 🏋️‍♀️ Weights & Biases
import wandb

# ⚡ 🤝 🏋️‍♀️
from pytorch_lightning.loggers import WandbLogger

##Configuracion

In [47]:
# Q-learning settings
learning_rate = 0.00025
discount_factor = 0.99
epochs = 20
learning_steps_per_epoch = 2000
replay_memory_size = 10000

# NN learning settings
batch_size = 64

# Training regime
test_episodes_per_epoch = 100

# Other parameters
frame_repeat = 12
resolution = (30, 45)
episodes_to_watch = 10

model_savefile = "./model-doom.pth"
save_model = True
load_model = False
skip_learning = False

# Configuration file path
config_file_path = "../scenarios/simpler_basic.cfg"
# config_file_path = "../../scenarios/rocket_basic.cfg"
# config_file_path = "../../scenarios/basic.cfg"

pl.seed_everything(hash("setting random seeds") % 2**32 - 1)

Global seed set to 537065132


537065132

##Preprocesamiento

In [40]:
# Converts and down-samples the input image
def preprocess(img):
    img = skimage.transform.resize(img, resolution)
    img = img.astype(np.float32)
    img = img.reshape([1, 1, resolution[0], resolution[1]])
    img = torch.from_numpy(img)
    img = Variable(img)
    return img

##Reproducir juego aprendido

In [27]:
class ReplayMemory:
    def __init__(self, capacity):
        channels = 1
        state_shape = (capacity, channels, resolution[0], resolution[1])
        self.s1 = np.zeros(state_shape, dtype=np.float32)
        self.s2 = np.zeros(state_shape, dtype=np.float32)
        self.a = np.zeros(capacity, dtype=np.int32)
        self.r = np.zeros(capacity, dtype=np.float32)
        self.isterminal = np.zeros(capacity, dtype=np.float32)

        self.capacity = capacity
        self.size = 0
        self.pos = 0

    def add_transition(self, s1, action, s2, isterminal, reward):
        self.s1[self.pos, 0, :, :] = s1
        self.a[self.pos] = action
        if not isterminal:
            self.s2[self.pos, 0, :, :] = s2
        self.isterminal[self.pos] = isterminal
        self.r[self.pos] = reward

        self.pos = (self.pos + 1) % self.capacity
        self.size = min(self.size + 1, self.capacity)

    def get_sample(self, sample_size):
        i = sample(range(0, self.size), sample_size)
        return self.s1[i], self.a[i], self.r[i], self.isterminal[i], self.s2[i] 

    def clean_memory(self):
        self.pos = 0
        self.size = 0



In [28]:
class RLDataset(IterableDataset):
    """
    Iterable Dataset containing the ReplayMemory
    which will be updated with new experiences during training
    Args:
        buffer: replay buffer
        sample_size: number of experiences to sample at a time
    """

    def __init__(self, buffer: ReplayMemory, sample_size: int = batch_size) -> None:
        self.buffer = buffer
        self.sample_size = sample_size

    def __iter__(self):
        states, actions, rewards, dones, new_states = self.buffer.get_sample(self.sample_size)
        for i in range(len(dones)):
            yield states[i], actions[i], rewards[i], dones[i], new_states[i]

##Modelo de Policy

In [29]:
class Net(nn.Module):
    def __init__(self, available_actions_count):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=6, stride=3)
        self.conv2 = nn.Conv2d(8, 8, kernel_size=3, stride=2)
        self.fc1 = nn.Linear(192, 128)
        self.fc2 = nn.Linear(128, available_actions_count)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, 192)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [56]:
class PolicyModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.lr      = learning_rate

        # Create Doom instance
        self.game = self.initialize_vizdoom(config_file_path)

        # Action = which buttons are pressed
        n = self.game.get_available_buttons_size()
        self.actions = [list(a) for a in it.product([0, 1], repeat=n)]

        # Create replay memory which will store the transitions
        self.memory = ReplayMemory(capacity=replay_memory_size)

        if load_model:
            print("Loading model from: ", model_savefile)
            self.net = torch.load(model_savefile)
        else:
            self.net = Net(len(self.actions))
        
    def initialize_vizdoom(self, config_file_path):
        game = DoomGame()
        game.load_config(config_file_path)
        game.set_window_visible(False)
        game.set_mode(Mode.PLAYER)
        game.set_screen_format(ScreenFormat.GRAY8)
        game.set_screen_resolution(ScreenResolution.RES_640X480)
        game.init()
        return game

    def forward(self, x): 
        logits = self.net(x)
        return Categorical(logits=logits)

    def get_action(self,state):
        return self(state).sample().item()
        
    def loss(self,state, action, reward):       
        logp = self(state).log_prob(action)
        return -(logp * reward).mean()

    def on_train_epoch_start(self):
        #Llena la memoria
        self.memory.clean_memory()
        self.game.new_episode()
        for learning_step in trange(learning_steps_per_epoch, leave=False):            
            state = preprocess(self.game.get_state().screen_buffer)
            action= self.get_action(state)
            reward = self.game.make_action(self.actions[action], frame_repeat)
            done = self.game.is_episode_finished()
            next_state = preprocess(self.game.get_state().screen_buffer) if not done else None
            # Remember the transition that was just experienced.
            self.memory.add_transition(state, action, next_state, done, reward)
            
            if self.game.is_episode_finished():
                score = self.game.get_total_reward()
                self.log('train/score', score, on_epoch=True)                
                self.game.new_episode()        

    def discount_reward(self,reward):
        R = 0
        rewards_with_discount=[]
        for r in torch.flip(reward, [0]):
            R = r + discount_factor * R
            rewards_with_discount.insert(0,R)
        
        # Scale rewards
        rewards_with_discount = torch.FloatTensor(rewards_with_discount)
        rewards_with_discount = (rewards_with_discount - rewards_with_discount.mean()) / (rewards_with_discount.std() + np.finfo(np.float32).eps)
        return rewards_with_discount

    def training_step(self, batch, batch_idx):
        state, action, reward, _, _ = batch

        # Discount future rewards back to the present       
        reward = self.discount_reward(reward) 
        
        # Calculate loss
        loss = self.loss(state,action,reward)        

        # logging metrics we calculated by hand
        self.log('train/loss', loss, on_epoch=True)
        
        return loss

    def configure_optimizers(self):
        return torch.optim.SGD(model.parameters(), self.lr)

    #paso de pruebas que se realizarán al finalizar todo el entrenamiento
    def test_step(self, batch, batch_idx):
        print("Hola")
        

    def on_validation_start(self):      
        for _ in trange(test_episodes_per_epoch, leave=False):
            self.game.new_episode()
            while not self.game.is_episode_finished():
                state = preprocess(self.game.get_state().screen_buffer)
                action= self.get_action(state)
                self.game.make_action(self.actions[action], frame_repeat)

            r = self.game.get_total_reward()
            self.log("val/total_score", r, on_step=True, on_epoch=True)

    def on_train_end(self):
        self.game.close()

        # Reinitialize the game with window visible
        self.game.set_window_visible(True)
        self.game.set_mode(Mode.ASYNC_PLAYER)
        self.game.init()

        for _ in range(episodes_to_watch):
            self.game.new_episode()
            while not self.game.is_episode_finished():
                state = preprocess(self.game.get_state().screen_buffer)
                action = self.get_action(state)

                # Instead of make_action(a, frame_repeat) in order to make the animation smooth
                self.game.set_action(self.actions[action])
                for _ in range(frame_repeat):
                    self.game.advance_action()

            # Sleep between episodes
            sleep(1.0)
            score = self.game.get_total_reward()
            print("test/total_score", score)

        torch.save(self.net, model_savefile)

    def train_dataloader(self) -> DataLoader:
        """Initialize the Replay Buffer dataset used for retrieving experiences"""
        self.hparams.episode_length = 200
        self.hparams.batch_size = 16
        dataset = RLDataset(self.memory, self.hparams.episode_length)
        dataloader = DataLoader(dataset=dataset,
                                batch_size=self.hparams.batch_size,
                                )
        return dataloader
            

In [57]:
!wandb login --relogin 70434303a5ed54f05105b62078db2447874ee020
wandb_logger = WandbLogger(project="VDoomLearning")

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\ruben/.netrc


In [58]:
trainer = pl.Trainer(
    logger = wandb_logger,    # W&B integration
    log_every_n_steps = 10,   # set the logging frequency
    gpus = 0,                # use all GPUs
    max_epochs = 1,           # number of epochs
    deterministic = True,     # keep it deterministic
    default_root_dir = "./"
)

model = PolicyModel()
trainer.fit(model)
wandb.finish()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
C:\Users\ruben\AppData\Local\Programs\Python\Python36\lib\site-packages\pytorch_lightning\loggers\wandb.py:342: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  "There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse"

  | Name | Type | Params
------------------------------
0 | net  | Net  | 26.6 K
------------------------------
26.6 K    Trainable params
0         Non-trainable params
26.6 K    Total params
0.106     Total estimated model params size (MB)
C:\Users\ruben\AppData\Local\Programs\Python\Python36\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory .\VDoomLearning\4cb1atpv\checkpoints exists and is not empty.
  rank_zer

Epoch 0: : 0it [00:00, ?it/s]

Epoch 0: : 13it [03:06, 14.38s/it, loss=-0.000171, v_num=atpv]
test/total_score 53.0
test/total_score 94.0
test/total_score 0.0
test/total_score 70.0
test/total_score 94.0
test/total_score -128.0
test/total_score -329.0
test/total_score -155.0
test/total_score -330.0
test/total_score -356.0


epoch,▁▁▁▁
train/loss_epoch,▁█
train/loss_step,▁█
train/score,█▁
trainer/global_step,▁█▁█
epoch,0
train/loss_epoch,0.00015
train/loss_step,0.00665
train/score,-100.21384
trainer/global_step,12


In [23]:
def learn(s1, target_q):
    s1 = torch.from_numpy(s1)
    target_q = torch.from_numpy(target_q)
    s1, target_q = Variable(s1), Variable(target_q)
    output = model(s1)
    loss = criterion(output, target_q)
    # compute gradient and do SGD step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss

def get_q_values(state):
    state = torch.from_numpy(state)
    state = Variable(state)
    return model(state)

def get_best_action(state):
    q = get_q_values(state)
    m, index = torch.max(q, 1)
    action = index.data.numpy()[0]
    return action


def learn_from_memory():
    """ Learns from a single transition (making use of replay memory).
    s2 is ignored if s2_isterminal """

    # Get a random minibatch from the replay memory and learns from it.
    if memory.size > batch_size:
        s1, a, s2, isterminal, r = memory.get_sample(batch_size)

        q = get_q_values(s2).data.numpy()
        q2 = np.max(q, axis=1)
        target_q = get_q_values(s1).data.numpy()
        # target differs from q only for the selected action. The following means:
        # target_Q(s,a) = r + gamma * max Q(s2,_) if isterminal else r
        target_q[np.arange(target_q.shape[0]), a] = r + discount_factor * (1 - isterminal) * q2
        learn(s1, target_q)


def perform_learning_step(epoch):
    """ Makes an action according to eps-greedy policy, observes the result
    (next state, reward) and learns from the transition"""

    def exploration_rate(epoch):
        """# Define exploration rate change over time"""
        start_eps = 1.0
        end_eps = 0.1
        const_eps_epochs = 0.1 * epochs  # 10% of learning time
        eps_decay_epochs = 0.6 * epochs  # 60% of learning time

        if epoch < const_eps_epochs:
            return start_eps
        elif epoch < eps_decay_epochs:
            # Linear decay
            return start_eps - (epoch - const_eps_epochs) / (eps_decay_epochs - const_eps_epochs) * (start_eps - end_eps)
        else:
            return end_eps

    s1 = preprocess(game.get_state().screen_buffer)

    # With probability eps make a random action.
    eps = exploration_rate(epoch)
    if random() <= eps:
        a = randint(0, len(actions) - 1)
    else:
        # Choose the best action according to the network.
        s1 = s1.reshape([1, 1, resolution[0], resolution[1]])
        a = get_best_action(s1)
    reward = game.make_action(actions[a], frame_repeat)

    isterminal = game.is_episode_finished()
    s2 = preprocess(game.get_state().screen_buffer) if not isterminal else None

    # Remember the transition that was just experienced.
    memory.add_transition(s1, a, s2, isterminal, reward)

    learn_from_memory()


# Creates and initializes ViZDoom environment.
def initialize_vizdoom(config_file_path):
    print("Initializing doom...")
    game = DoomGame()
    game.load_config(config_file_path)
    game.set_window_visible(False)
    game.set_mode(Mode.PLAYER)
    game.set_screen_format(ScreenFormat.GRAY8)
    game.set_screen_resolution(ScreenResolution.RES_640X480)
    game.init()
    print("Doom initialized.")
    return game


if __name__ == '__main__':
    # Create Doom instance
    game = initialize_vizdoom(config_file_path)

    # Action = which buttons are pressed
    n = game.get_available_buttons_size()
    actions = [list(a) for a in it.product([0, 1], repeat=n)]

    # Create replay memory which will store the transitions
    memory = ReplayMemory(capacity=replay_memory_size)

    if load_model:
        print("Loading model from: ", model_savefile)
        model = torch.load(model_savefile)
    else:
        model = Net(len(actions))
    
    optimizer = torch.optim.SGD(model.parameters(), learning_rate)

    print("Starting the training!")
    time_start = time()
    if not skip_learning:
        for epoch in range(epochs):
            print("\nEpoch %d\n-------" % (epoch + 1))
            train_episodes_finished = 0
            train_scores = []

            print("Training...")
            self.game.new_episode()
            for learning_step in trange(learning_steps_per_epoch, leave=False):
                perform_learning_step(epoch)
                if game.is_episode_finished():
                    score = game.get_total_reward()
                    train_scores.append(score)
                    game.new_episode()
                    train_episodes_finished += 1

            print("%d training episodes played." % train_episodes_finished)

            train_scores = np.array(train_scores)

            print("Results: mean: %.1f +/- %.1f," % (train_scores.mean(), train_scores.std()), \
                  "min: %.1f," % train_scores.min(), "max: %.1f," % train_scores.max())

            print("\nTesting...")
            test_episode = []
            test_scores = []
            for test_episode in trange(test_episodes_per_epoch, leave=False):
                game.new_episode()
                while not game.is_episode_finished():
                    state = preprocess(game.get_state().screen_buffer)
                    state = state.reshape([1, 1, resolution[0], resolution[1]])
                    best_action_index = get_best_action(state)

                    game.make_action(actions[best_action_index], frame_repeat)
                r = game.get_total_reward()
                test_scores.append(r)

            test_scores = np.array(test_scores)
            print("Results: mean: %.1f +/- %.1f," % (
                test_scores.mean(), test_scores.std()), "min: %.1f" % test_scores.min(),
                  "max: %.1f" % test_scores.max())

            print("Saving the network weigths to:", model_savefile)
            torch.save(model, model_savefile)

            print("Total elapsed time: %.2f minutes" % ((time() - time_start) / 60.0))

    game.close()
    print("======================================")
    print("Training finished. It's time to watch!")

    # Reinitialize the game with window visible
    game.set_window_visible(True)
    game.set_mode(Mode.ASYNC_PLAYER)
    game.init()

    for _ in range(episodes_to_watch):
        game.new_episode()
        while not game.is_episode_finished():
            state = preprocess(game.get_state().screen_buffer)
            state = state.reshape([1, 1, resolution[0], resolution[1]])
            best_action_index = get_best_action(state)

            # Instead of make_action(a, frame_repeat) in order to make the animation smooth
            game.set_action(actions[best_action_index])
            for _ in range(frame_repeat):
                game.advance_action()

        # Sleep between episodes
        sleep(1.0)
        score = game.get_total_reward()
        print("Total score: ", score)


Initializing doom...
Doom initialized.
Starting the training!

Epoch 1
-------
Training...


165 training episodes played.
Results: mean: -95.6 +/- 186.4, min: -380.0, max: 95.0,

Testing...


Results: mean: -70.4 +/- 233.2, min: -410.0 max: 95.0
Saving the network weigths to: ./model-doom.pth
Total elapsed time: 7.18 minutes

Epoch 2
-------
Training...


181 training episodes played.
Results: mean: -68.4 +/- 169.9, min: -385.0, max: 95.0,

Testing...


Results: mean: -41.2 +/- 218.8, min: -410.0 max: 95.0
Saving the network weigths to: ./model-doom.pth
Total elapsed time: 12.59 minutes

Epoch 3
-------
Training...


...

Results: mean: 88.3 +/- 10.6, min: 52.0 max: 95.0
Saving the network weigths to: ./model-doom.pth
Total elapsed time: 101.93 minutes

Epoch 19
-------
Training...


1001 training episodes played.
Results: mean: 82.0 +/- 17.9, min: -28.0, max: 95.0,

Testing...


Results: mean: 85.0 +/- 13.0, min: 52.0 max: 95.0
Saving the network weigths to: ./model-doom.pth
Total elapsed time: 107.92 minutes

Epoch 20
-------
Training...


1035 training episodes played.
Results: mean: 82.7 +/- 17.9, min: -43.0, max: 95.0,

Testing...


Results: mean: 86.9 +/- 12.3, min: 45.0 max: 95.0
Saving the network weigths to: ./model-doom.pth
Total elapsed time: 113.88 minutes
Training finished. It's time to watch!
Total score:  70.0
Total score:  82.0
Total score:  94.0
Total score:  12.0
Total score:  94.0
Total score:  94.0
Total score:  94.0
Total score:  95.0
Total score:  70.0
Total score:  60.0
